**Step 1: Import packages**

In [20]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
import tensorflow as tf

# sklearn libraries for preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# sklearn libraries for models
from sklearn.linear_model import LogisticRegression

# sklearn libraries for evaluation
from sklearn import metrics

**Step 2: Define working directories**

In [2]:
os.chdir('C:/Users/user/Documents/Berkeley_MIDS/W207_Machine_Learning/w207_final_project')

**Step 3: Define classes**

**Step 4: Define functions**

In [3]:
def reduce(X, n):
    '''PCA dimensionality reduction for X array and n principal components'''
    pca = PCA(n_components = n) 
    X_reduced = pca.fit_transform(X)
    return X_reduced

In [4]:
def data_split(X, y, test_p, mini_s):
    '''Split train and test data for X,y data arrays with test_p percentage of observations in the test data and 
    mini_s number of observations in the mini_train set.'''
    # Split into train and test data
    train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=test_p, random_state=1, stratify=y)

    # Split test data into development data and training into mini train set for computation time
    dev_data, dev_labels = test_data[:int(test_data.shape[0]*0.5)], test_labels[:int(test_data.shape[0]*0.5)]
    mini_train_data, mini_train_labels = train_data[:mini_s], train_labels[:mini_s]
    print('Train data shape:', train_data.shape)
    print('Dev data shape:', dev_data.shape)
    print('Mini_train data shape:', mini_train_data.shape)
    
    return train_data, train_labels, test_data, test_labels, dev_data, dev_labels, mini_train_data, mini_train_labels

In [5]:
def log_reg(train_X, train_Y, dev_X, dev_Y):
    '''Logistic regression looping through values of C. Input train and test data and labels and funtion 
    returns list of accuracies and list of C values.'''
    c_values = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0, 2.0, 5.0]
    LR = []
    for c in c_values:
        clf = LogisticRegression(C=c, solver='liblinear', multi_class='auto')
        clf.fit(train_X, train_Y)
        clf_pred = clf.predict(dev_X)
        LR.append(round(metrics.f1_score(dev_Y, clf_pred, average='weighted'),3))

    return LR, c_values

In [59]:
def LR_accuracy_c(train_X, train_Y, dev_X, dev_Y):
    '''Calls logistic regression function and returns highest accuracy for specific value of C.'''
    accuracy_list = log_reg(train_X, train_Y, dev_X, dev_Y)[0]
    accuracy = [y for x,y in sorted(enumerate(accuracy_list), key = lambda x: x[1], reverse=True)][:1]
    accuracy_index = [x for x,y in sorted(enumerate(accuracy_list), key = lambda x: x[1], reverse=True)][:1]
    C = log_reg(train_X, train_Y, dev_X, dev_Y)[1][accuracy_index[0]]
    
    return accuracy[0], C

**Step 5: Read data**

5.1 Load and combine pixel data files into dataframe

In [7]:
# Load npz files to combine and print content files
data_1 = np.load('et_w207_project_npz_files_5000_tmp_tmp5ocrhjnn.npz')
data_1.close

data_2 = np.load('et_w207_project_npz_files_5000_tmp_tmp092sag67.npz')
data_2.close

data_3 = np.load('et_w207_project_npz_files_5000_tmp_tmpf31_pn8p.npz')
data_3.close

data_4 = np.load('et_w207_project_npz_files_5000_tmp_tmpq5b2g4n2.npz')
data_4.close

print(data_1.files)

['arr_0', 'arr_1']


In [8]:
# Show file arr_0 example
data_1['arr_0'][:5]

array(['ozbpir-9ey6js-ggdqwo', '378ykanq', 'p366md-big834-7o23k6',
       '3zruwvl2', 'q18iae-3vnh74-79npmy'], dtype='<U20')

In [9]:
# Show file arr_1 example
data_1['arr_1'][:5]

array([[-103.939   , -116.779   , -123.68    , ..., -103.939   ,
        -116.779   , -123.68    ],
       [ -46.939003,  -74.779   , -118.68    , ...,  -57.939003,
         -84.779   , -118.68    ],
       [ 151.061   ,  138.22101 ,  131.32    , ...,  151.061   ,
         138.22101 ,  131.32    ],
       [  -3.939003,  -66.779   ,  -96.68    , ...,   -3.939003,
         -66.779   ,  -96.68    ],
       [-103.939   , -116.779   , -123.68    , ..., -103.939   ,
        -116.779   , -123.68    ]], dtype=float32)

In [10]:
# Get number of observations in each file
print(data_1['arr_0'].shape)
print(data_2['arr_0'].shape)
print(data_3['arr_0'].shape)
print(data_4['arr_0'].shape)

(878,)
(893,)
(638,)
(1353,)


In [11]:
# Concatenate data from all files
data_key = np.concatenate((data_1['arr_0'], data_2['arr_0'], data_3['arr_0'], data_4['arr_0']))

data_pixel = np.concatenate((data_1['arr_1'], data_2['arr_1'], data_3['arr_1'], data_4['arr_1']))
print(data_pixel.shape)

(3762, 150528)


In [12]:
# Create key value pairs with attachment key and pixel array
# Standardize pixel values between 0 and 1 by dividing by 255
# Store pixel data as dataframe
data_dict = {}
for i in range(data_key.shape[0]):
    data_dict[data_key[i]] = [data_pixel[i]/255]

df_pixel = pd.DataFrame(data=data_dict)
df_pixel_t = df_pixel.transpose()
df_pixel_t.index.names = ['attachment_key']
df_pixel_t.columns = ['pixel_array']
df_pixel_t

,pixel_array
attachment_key,
ozbpir-9ey6js-ggdqwo,"[-0.40760392, -0.45795685, -0.4850196, -0.4076..."
378ykanq,"[-0.18407452, -0.29325098, -0.46541175, -0.097..."
p366md-big834-7o23k6,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
3zruwvl2,"[-0.015447071, -0.26187843, -0.37913725, -0.01..."
q18iae-3vnh74-79npmy,"[-0.40760392, -0.45795685, -0.4850196, -0.4076..."
...,...
pktyig-481wyg-6gyulf,"[-0.3605451, -0.41089803, -0.38305882, -0.3644..."
zu7y3rha,"[-0.007603933, -0.05795686, -0.08501961, -0.00..."
pxcwxf-8tnpyg-8vhb9r,"[0.3022, 0.18125883, 0.12674509, 0.31004313, 0..."


5.2 Load metadata file

In [34]:
# Load excel data
df = pd.read_csv('school_project_data_set-new.csv')
df.head()

,attachment_key,size_mb,height,width,h_to_w,filename,logo
0,001lq61k,0.122520,514.0,720.0,0.713889,campus ministry on the beach.jpg,0
1,002dlirq,3.256944,4072.0,3868.0,1.052741,BIG ALS FISH FLASH UV VP PL.jpg,0
2,003qefgm,0.002522,80.0,200.0,0.400000,netix_email_studio.png,1
3,0083okjl,0.116802,600.0,600.0,1.000000,productshot2.jpg,0
4,00a957mh,0.140714,816.0,2382.0,0.342569,divine_medical_billing_inc_master.png,1


5.3 Combine pixel and metadata features in a pandas dataframe

In [35]:
# Merge dataframes by attachment key
combined_df = df.merge(df_pixel_t, on='attachment_key')
combined_df

,attachment_key,size_mb,height,width,h_to_w,filename,logo,pixel_array
0,00xjny6u,0.011849,520.0,506.0,1.027668,Esterdale Theatre - Logo.png,1,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
1,04o31jop,0.012610,269.0,396.0,0.679293,BLUE_LOGO.png,1,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
2,0840h7ox,0.399114,518.0,920.0,0.563044,ZEN_LB-5.jpg,0,"[0.106121555, 0.05576863, 0.052235294, 0.09043..."
3,0awudx2h,0.011226,200.0,200.0,1.000000,article central 200.jpg,1,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
4,0bb8y85h,0.263813,885.0,800.0,1.106250,1116-100_WGTank_white.jpg,0,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
...,...,...,...,...,...,...,...,...
3750,zu7y3rha,0.147371,498.0,702.0,0.709402,journalism.jpg,0,"[-0.007603933, -0.05795686, -0.08501961, -0.00..."
3751,zuanbsqe,0.580392,206.0,415.0,0.496386,2013 05 26_techstars_id_final_bug solo graysca...,1,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
3752,zw9234v4,2.021135,1645.0,1450.0,1.134483,SPIN N GLO SILVER WING GPK MY.jpg,0,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,..."
3753,zxfqbfj1,0.012057,36.0,345.0,0.104348,cornerstone web logo medium.png,1,"[-0.40760392, -0.45795685, -0.4850196, -0.4076..."


In [29]:
### VISUALIZATIONS FROM PANDAS PROFILING HERE

**Step 6: Data preprocessing**

6.1 Remove null values

In [36]:
# Remove rows with null values
combined_df = combined_df[combined_df.height.notnull()]
combined_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3754 entries, 0 to 3754
Data columns (total 8 columns):
attachment_key    3754 non-null object
size_mb           3754 non-null float64
height            3754 non-null float64
width             3754 non-null float64
h_to_w            3754 non-null float64
filename          3754 non-null object
logo              3754 non-null int64
pixel_array       3754 non-null object
dtypes: float64(4), int64(1), object(3)
memory usage: 264.0+ KB


6.2 Create file type dummy variables from file name

In [37]:
# Find file types from filename
file_types = []
for item in combined_df['filename']:
    file_types.append(item[-3:].lower())

print(set(file_types))

{'73e', 'peg', 'jpg', 'png', 'ack', 'age', 'hot', 'ite', 'pig', '0mm', 'lue', 'jpe'}


In [38]:
# Create list of file types from filename to append to df
filetype_encoding = []
for item in combined_df['filename']:
    if item[-3:].lower() == 'jpg':
        filetype_encoding.append('jpg')
    elif item[-3:].lower() == 'png':
        filetype_encoding.append('png')
    else:
        filetype_encoding.append('other')

In [41]:
# Create df that includes column for file type
combined_df['filetype'] = filetype_encoding
combined_df = combined_df[['attachment_key', 'size_mb', 'height', 'width', 'h_to_w', 'filetype', 'pixel_array', 'filename','logo']]
combined_df.head()
#warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

,attachment_key,size_mb,height,width,h_to_w,filetype,pixel_array,filename,logo
0,00xjny6u,0.011849,520.0,506.0,1.027668,png,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",Esterdale Theatre - Logo.png,1
1,04o31jop,0.012610,269.0,396.0,0.679293,png,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",BLUE_LOGO.png,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,jpg,"[0.106121555, 0.05576863, 0.052235294, 0.09043...",ZEN_LB-5.jpg,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",article central 200.jpg,1
4,0bb8y85h,0.263813,885.0,800.0,1.106250,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",1116-100_WGTank_white.jpg,0


In [42]:
# Get dummies for file type
filetype_dummies = pd.get_dummies(combined_df.filetype)

# add to initial df
combined_df = pd.concat((combined_df, filetype_dummies), axis=1)

# print head of df
combined_df.head()

,attachment_key,size_mb,height,width,h_to_w,filetype,pixel_array,filename,logo,jpg,other,png
0,00xjny6u,0.011849,520.0,506.0,1.027668,png,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",Esterdale Theatre - Logo.png,1,0,0,1
1,04o31jop,0.012610,269.0,396.0,0.679293,png,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",BLUE_LOGO.png,1,0,0,1
2,0840h7ox,0.399114,518.0,920.0,0.563044,jpg,"[0.106121555, 0.05576863, 0.052235294, 0.09043...",ZEN_LB-5.jpg,0,1,0,0
3,0awudx2h,0.011226,200.0,200.0,1.000000,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",article central 200.jpg,1,1,0,0
4,0bb8y85h,0.263813,885.0,800.0,1.106250,jpg,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",1116-100_WGTank_white.jpg,0,1,0,0


6.3 Scale metadata variables

In [43]:
# Scale continuous feature variables
combined_df['size_mb'] = preprocessing.scale(combined_df['size_mb'])
combined_df['h_to_w'] = preprocessing.scale(combined_df['h_to_w'])
combined_df['height'] = preprocessing.scale(combined_df['height'])
combined_df['width'] = preprocessing.scale(combined_df['width'])

6.4 Create X, y arrays

In [44]:
# Organize df to separate into feature arrays
combined_df = combined_df[['attachment_key', 'pixel_array', 'size_mb', 'h_to_w', 'jpg', 'png', 'other',
                                     'height', 'width', 'logo', 'filetype', 'filename']]
combined_df.head()

,attachment_key,pixel_array,size_mb,h_to_w,jpg,png,other,height,width,logo,filetype,filename
0,00xjny6u,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",-0.366308,0.638439,0,1,0,-0.712708,-0.888611,1,png,Esterdale Theatre - Logo.png
1,04o31jop,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",-0.366187,-0.187892,0,1,0,-0.861519,-0.943446,1,png,BLUE_LOGO.png
2,0840h7ox,"[0.106121555, 0.05576863, 0.052235294, 0.09043...",-0.304882,-0.463630,1,0,0,-0.713894,-0.682232,0,jpg,ZEN_LB-5.jpg
3,0awudx2h,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",-0.366407,0.572811,1,0,0,-0.902427,-1.041152,1,jpg,article central 200.jpg
4,0bb8y85h,"[0.5923961, 0.54204315, 0.51498044, 0.5923961,...",-0.326343,0.824832,1,0,0,-0.496310,-0.742052,0,jpg,1116-100_WGTank_white.jpg


In [45]:
# y array contains logo/not logo binary variable
y = np.array(combined_df.iloc[:, 9])

# X array contains pixel array only
X = np.array(combined_df['pixel_array'].tolist())
print(X.shape)

(3754, 150528)


6.5 Reduce dimensionality of pixel data

In [46]:
# Reduce dimensionality of pixel array
X_reduced = reduce(X,2)

In [ ]:
### PCA VISUALIZATIONS HERE

6.6 Create alternate X arrays with metadata features

In [47]:
# Create variations of X array with reduced pixel data and different metadata features

# X2 array contains pixel features, size, height, width, h_to_w, filetype dummies
X2 = np.concatenate((X_reduced, np.array(combined_df.iloc[:, 2:9])),axis=1)
print(X2.shape)

# X3 array contains pixel features, size, h_to_w, filetype dummies
X3 = np.concatenate((X_reduced, np.array(combined_df.iloc[:, 2:7])),axis=1)
print(X3.shape)

# X4 array contains pixel array, size, h_to_w
X4 = np.concatenate((X_reduced, np.array(combined_df.iloc[:, 2:4])),axis=1)
print(X4.shape)

# X5 array contains pixel array, filetype dummies 
X5 = np.concatenate((X_reduced, np.array(combined_df.iloc[:, 4:7])),axis=1)
print(X5.shape)

# X6 array contains size, height, width, h_to_w, filetype
X6 = np.array(combined_df.iloc[:, 2:9])
print(X6.shape)

# X7 array contains size, h_to_w, filetype
X7 = np.array(combined_df.iloc[:, 2:7])
print(X7.shape)

(3754, 9)
(3754, 7)
(3754, 4)
(3754, 5)
(3754, 7)
(3754, 5)


6.7 Split each set of features into train, development, and test data

In [48]:
# Split all feature combinations into train, test data
train_data, train_labels, test_data, test_labels, dev_data, dev_labels, mini_train_data, mini_train_labels = data_split(X_reduced, y, 0.4, 100)
train_data2, train_labels2, test_data2, test_labels2, dev_data2, dev_labels2, mini_train_data2, mini_train_labels2 = data_split(X2, y, 0.4, 100)
train_data3, train_labels3, test_data3, test_labels3, dev_data3, dev_labels3, mini_train_data3, mini_train_labels3 = data_split(X3, y, 0.4, 100)
train_data4, train_labels4, test_data4, test_labels4, dev_data4, dev_labels4, mini_train_data4, mini_train_labels4 = data_split(X4, y, 0.4, 100)
train_data5, train_labels5, test_data5, test_labels5, dev_data5, dev_labels5, mini_train_data5, mini_train_labels5 = data_split(X5, y, 0.4, 100)
train_data6, train_labels6, test_data6, test_labels6, dev_data6, dev_labels6, mini_train_data6, mini_train_labels6 = data_split(X6, y, 0.4, 100)
train_data7, train_labels7, test_data7, test_labels7, dev_data7, dev_labels7, mini_train_data7, mini_train_labels7 = data_split(X7, y, 0.4, 100)

Train data shape: (2252, 2)
Dev data shape: (751, 2)
Mini_train data shape: (100, 2)
Train data shape: (2252, 9)
Dev data shape: (751, 9)
Mini_train data shape: (100, 9)
Train data shape: (2252, 7)
Dev data shape: (751, 7)
Mini_train data shape: (100, 7)
Train data shape: (2252, 4)
Dev data shape: (751, 4)
Mini_train data shape: (100, 4)
Train data shape: (2252, 5)
Dev data shape: (751, 5)
Mini_train data shape: (100, 5)
Train data shape: (2252, 7)
Dev data shape: (751, 7)
Mini_train data shape: (100, 7)
Train data shape: (2252, 5)
Dev data shape: (751, 5)
Mini_train data shape: (100, 5)


**Step 7: Logistic Regression**

7.1 Run logistic regression for pixel data only

In [60]:
# Pixel array only
# Run LR and output best accuracy with c value
print('Features: pixel features only\nBest Accuracy: {}, c={}'.format(
    LR_accuracy_c(train_data, train_labels, dev_data, dev_labels)[0],
    LR_accuracy_c(train_data, train_labels, dev_data, dev_labels)[1]))

Features: pixel features only
Best Accuracy: 0.577, c=0.001


7.2 Run logistic regression for all feature combinations

In [61]:
# Run LR and output best accuracies with c values for each combination of features
print('Features: pixel features, size, height, width, h_to_w, filetype dummies\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data2, train_labels2, dev_data2, dev_labels2)[0],
    LR_accuracy_c(train_data2, train_labels2, dev_data2, dev_labels2)[1]))

print('Features: pixel features, size, h_to_w, filetype dummies\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data3, train_labels3, dev_data3, dev_labels3)[0],
    LR_accuracy_c(train_data3, train_labels3, dev_data3, dev_labels3)[1]))

print('Features: pixel features, size, h_to_w\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data4, train_labels4, dev_data4, dev_labels4)[0],
    LR_accuracy_c(train_data4, train_labels4, dev_data4, dev_labels4)[1]))

print('Features: pixel features, filetype dummies\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data5, train_labels5, dev_data5, dev_labels5)[0],
    LR_accuracy_c(train_data5, train_labels5, dev_data5, dev_labels5)[1]))

print('Features: size, height, width, h_to_w, filetype\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data6, train_labels6, dev_data6, dev_labels6)[0],
    LR_accuracy_c(train_data6, train_labels6, dev_data6, dev_labels6)[1]))

print('Features: size, h_to_w, filetype\nBest Accuracy: {}, c={}\n'.format(
    LR_accuracy_c(train_data7, train_labels7, dev_data7, dev_labels7)[0],
    LR_accuracy_c(train_data7, train_labels7, dev_data7, dev_labels7)[1]))

Features: pixel features, size, height, width, h_to_w, filetype dummies
Best Accuracy: 0.858, c=0.1

Features: pixel features, size, h_to_w, filetype dummies
Best Accuracy: 0.849, c=2.0

Features: pixel features, size, h_to_w
Best Accuracy: 0.768, c=0.3

Features: pixel features, filetype dummies
Best Accuracy: 0.822, c=0.2

Features: size, height, width, h_to_w, filetype
Best Accuracy: 0.845, c=0.1

Features: size, h_to_w, filetype
Best Accuracy: 0.849, c=0.01



**Step 8: Multilayer NN**

**Conclusions**